In [1]:
import numpy as np

q4_data = np.load('q4_data/q4_data.npy', allow_pickle=True).item()
x_train = q4_data['q4x_train']
y_train = q4_data['q4y_train'].flatten()
x_test = q4_data['q4x_test']
y_test = q4_data['q4y_test'].flatten()

In [2]:
# Hyperparameters
lr = 1e-3
C = 1
num_epochs = 2000

print('Number of training data:', x_train.shape[0])
print('Number of test data    :', x_test.shape[0])
print('Feature dimension      :', x_test.shape[1])

Number of training data: 76
Number of test data    : 24
Feature dimension      : 4


In [3]:
def svm_train_bgd(x, y, C, lr, num_epochs):
    N, D = x.shape
    w = np.zeros(D)
    b = 0
    
    for i in range(num_epochs):
        indicator = 1 - y*(x@w + b)
        indicator[indicator < 0] = 0
        indicator[indicator > 0] = 1
        w_grad = w - C * x.T @ (indicator * y) 
        b_grad = -C * np.sum(indicator * y)
        w = w -  lr * w_grad 
        b = b -  lr * b_grad      
    
    return w, b

In [4]:
def svm_train_sgd(x, y, C, lr, num_epochs):
    N, D = x.shape
    w = np.zeros(D)
    b = 0

    for i in range(num_epochs):
        for j in range(N):
            indicator = 1 - y[j] * (w@x[j] + b)
            indicator = 1 if indicator > 0 else 0
            w_grad = (1/N) * w - C * indicator * y[j] * x[j]
            b_grad = -C * indicator * y[j]
            w = w - lr * w_grad  
            b = b - lr * b_grad
    
    return w, b

In [5]:
def svm_predict(x, w, b):

    y_pred = np.ones(x.shape[0])    
    result = x@w + b
    y_pred[result < 0] = -1

    return y_pred

In [6]:
def evaluate(preds, labels):

    return (preds == labels).mean()

In [7]:
# Train SVM with BGD
w, b = svm_train_bgd(x_train, y_train, C, lr, num_epochs)
print('Weights', w)
print('Bias   ', b)
 
# Test accuracy
preds_test = svm_predict(x_test, w, b)
test_acc = evaluate(preds_test, y_test)
print('BGD test accuracy: {:5.2f}%'.format(100.*test_acc))

Weights [-0.18558916 -0.33076923  0.85189816  0.70158907]
Bias    -0.7170000000000005
BGD test accuracy: 95.83%


In [8]:
# Train SVM with SGD
w, b = svm_train_sgd(x_train, y_train, C, lr, num_epochs)
print('Weights', w)
print('Bias   ', b)

# Test accuracy
preds_test = svm_predict(x_test, w, b)
test_acc = evaluate(preds_test, y_test)
print('SGD test accuracy: {:5.2f}%'.format(100.*test_acc))

Weights [-0.18895699 -0.34671336  0.84538791  0.76344282]
Bias    -0.7350000000000005
SGD test accuracy: 95.83%
